# Understanding retries
Here description of what this does.

In [1]:
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
import re

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)
jobs = {}
jobgroups = {} # key=initial_id, val= status, walltime, depth, failedattempts, core, errorcode, processingtype, #totalfailedattempts, sharedjob

status_order = {
    "finished"  : 0,
    "merging"   : 1,
    "cancelled" : 2,
    "failed"    : 3,
    "closed"    : 4,
    "undefined" : 5
}

In [2]:
def updatechain(currentid, depth, initialid):
    ### Update job chain information    
    
    global jobs
    global jobgroups
    global testid

    if currentid not in jobs:
        return None
        
    if jobs[currentid][10] >= 0:
        jobgroups[initialid][8] = True
        jobgroups[jobs[currentid][10]][8] = True
    jobs[currentid][10] = initialid
    if jobs[currentid][6] == "failed":
        depth += 1
        jobgroups[initialid][7] += 1
    maxdepthid = (depth, currentid)
        
    jobgroups[initialid][1] += jobs[currentid][5]
    if status_order[jobs[currentid][6]] < status_order[jobgroups[initialid][0]]:
        jobgroups[initialid][0] = jobs[currentid][6]
    for child in jobs[currentid][4]:
        tmpdepthid = updatechain(child, depth, initialid)
        if tmpdepthid != None and tmpdepthid[0] > maxdepthid[0]:
            maxdepthid = tmpdepthid
    return maxdepthid

### select period

In [3]:
start_date = datetime.datetime(2017, 8, 1)
#curre_date = datetime.datetime(2017, 8, 10, 23, 59, 59)
curre_date = datetime.datetime(2017, 8, 31, 23, 59, 59)
#curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days
timemargin = datetime.timedelta(1)

### selecting indices to scan

In [4]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-08-01,jobs_archive_2017-08-02,jobs_archive_2017-08-03,jobs_archive_2017-08-04,jobs_archive_2017-08-05,jobs_archive_2017-08-06,jobs_archive_2017-08-07,jobs_archive_2017-08-08,jobs_archive_2017-08-09,jobs_archive_2017-08-10,jobs_archive_2017-08-11,jobs_archive_2017-08-12,jobs_archive_2017-08-13,jobs_archive_2017-08-14,jobs_archive_2017-08-15,jobs_archive_2017-08-16,jobs_archive_2017-08-17,jobs_archive_2017-08-18,jobs_archive_2017-08-19,jobs_archive_2017-08-20,jobs_archive_2017-08-21,jobs_archive_2017-08-22,jobs_archive_2017-08-23,jobs_archive_2017-08-24,jobs_archive_2017-08-25,jobs_archive_2017-08-26,jobs_archive_2017-08-27,jobs_archive_2017-08-28,jobs_archive_2017-08-29,jobs_archive_2017-08-30,jobs_archive_2017-08-31


### running the scan

In [5]:
job_query = {
    "size": 0,
    "_source": [ "corecount", "exeerrorcode", "pandaid", "failedattempt",
                "child_ids", "wall_time", "jobstatus", "processingtype", "actualcorecount",
                "creationtime", "endtime"],
    'query':{
#        "match_all": {}
             'bool':{
#                    'must':[
#                         { "term": {"jobstatus": "finished" } },
#                         { "term": {"prodsourcelabel": "managed" } }
#                     ],
#                    'should':[
                       # { "term": {"prodsourcelabel": "rc_test" } },
                       # { "term": {"prodsourcelabel": "prod_test" } },
                       # { "term": {"prodsourcelabel": "managed" } }
#                        { "term": {"jobstatus": "finished" } },
#                        { "term": {"jobstatus": "failed" } },
#                    ]
             }
    }
                
}

scroll = scan(client=es, index=job_indices, query=job_query, scroll='1000000m', timeout="1000000m", size=10000)

count = 0
for res in scroll:
    count += 1
    
    #print(res)
    #if count>10000: break
    
    r = res['_source']
    
    cores = r['corecount']
    errorcode = r['exeerrorcode']
    pandaid = r['pandaid']
    failedattempt = r['failedattempt']
    if 'child_ids' in r:
        child_ids = r['child_ids']
    else:
        child_ids=[]
    wall_time = r['wall_time']
    jobstatus = r['jobstatus']
    processingtype = r['processingtype']
    actualcorecount = r['actualcorecount']
    creationtime = r['creationtime']
    endtime = r['endtime']
    
    if actualcorecount != None:
        wall_time *= actualcorecount
    elif cores != None:
        wall_time *= cores
    
    if not count%100000: 
        print(count)
        #print(data)
    
    jobs[pandaid]=[True, cores, errorcode, failedattempt, child_ids, wall_time, jobstatus, processingtype, creationtime, endtime, -1]
    #data.append(doc)

#job=pd.DataFrame(data,columns=['initialjob', 'cores', 'errorcode', 'pandaid', 'failedattempt', 'child_ids', 'wall_time', 'jobstatus', 'processingtype'])
#del data
#job.head(100)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

### Debug

In [2]:
'''
jobs.clear()
jobs[0]=[True, 1, 0, 0, [1], 1, "failed", "debug"]
jobs[1]=[True, 1, 0, 0, [2], 1, "failed", "debug"]
jobs[2]=[True, 1, 0, 0, [3,5], 1, "failed", "debug"]
jobs[3]=[True, 1, 0, 0, [4], 1, "failed", "debug"]
jobs[4]=[True, 1, 0, 0, [], 1, "finished", "debug"]
jobs[5]=[True, 1, 0, 0, [6], 1, "failed", "debug"]
jobs[6]=[True, 1, 0, 0, [7], 1, "failed", "debug"]
jobs[7]=[True, 1, 0, 0, [], 1, "failed", "debug"]
print(jobs)
'''

'\njobs.clear()\njobs[0]=[True, 1, 0, 0, [1], 1, "failed", "debug"]\njobs[1]=[True, 1, 0, 0, [2], 1, "failed", "debug"]\njobs[2]=[True, 1, 0, 0, [3,5], 1, "failed", "debug"]\njobs[3]=[True, 1, 0, 0, [4], 1, "failed", "debug"]\njobs[4]=[True, 1, 0, 0, [], 1, "finished", "debug"]\njobs[5]=[True, 1, 0, 0, [6], 1, "failed", "debug"]\njobs[6]=[True, 1, 0, 0, [7], 1, "failed", "debug"]\njobs[7]=[True, 1, 0, 0, [], 1, "failed", "debug"]\nprint(jobs)\n'

### Find initial jobs

In [7]:
count_child = 0
count_parent = 0
for key,job in jobs.items():
    if len(job[4]) > 1 and job[7] == "simul":
        count_child += 1
        if (count_child < 0):
            print('Multi-child : '+str(key))

    for child in job[4]:
        if child in jobs:
            if jobs[child][0] == False:
                count_parent += 1
                if (count_parent < 100):
                    print('Multi-parent : '+str(key)+' '+str(child)+job[7])
            jobs[child][0] = False


Multi-parent : 3531473125 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473129 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473135 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473143 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473148 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473152 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473156 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473160 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473168 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473172 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473175 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473181 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473186 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473192 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473198 3542149881panda-client-0.5.85-jedi-run
Multi-parent : 3531473203

### Make a jobgroup from starting from an initial job

In [8]:
for key,job in jobs.items():
    #jobstartstr = re.sub('\d+(st|nd|rd|th)', lambda m: m.group()[:-2].zfill(2), job[8])
    #jobendstr   = re.sub('\d+(st|nd|rd|th)', lambda m: m.group()[:-2].zfill(2), job[9])
    jobstart = datetime.datetime.strptime(job[8], '%Y-%m-%dT%X')
    jobend   = datetime.datetime.strptime(job[9], '%Y-%m-%dT%X')
    if job[0] == True and \
       jobstart > start_date + timemargin and jobend > start_date + timemargin and \
       jobstart < curre_date - timemargin and jobend < curre_date - timemargin:
           jobgroups[key] = ["undefined", 0, 0, 0, job[1], job[2], job[7], 0, False]
print('Number of job groups')
print(len(jobgroups))

Number of job groups
31737717


### Follow job chains

In [9]:
count = 0
for key in jobgroups.keys():
    maxdepthid = updatechain(key, 0, key)
    if maxdepthid != None:
        jobgroups[key][2] = maxdepthid[0]
        if maxdepthid[0] in jobs:
            jobgroups[key][3] = jobs[maxdepthid[0]][3]
    if not count%100000: 
        print(count)
        print(jobgroups[key])
    count += 1
print('end')        

0
['finished', 342, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
100000
['finished', 3681, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-athena-trf', 0, False]
200000
['finished', 216, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
300000
['finished', 133, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
400000
['closed', 0, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-athena-trf', 0, False]
500000
['finished', 179, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
600000
['finished', 361, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-athena', 0, False]
700000
['finished', 662, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
800000
['finished', 2661, 0, 0, 1, 0, 'pmerge', 0, False]
900000
['finished', 123240, 0, 0, 8, 0, 'pile', 0, False]
1000000
['finished', 335, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-athena', 0, False]
1100000
['cancelled', 618, 2, 0, 1, 0, 'panda-client-0.5.86-jedi-athena', 2, False]
1200000
['finished', 255, 0, 0, 1, 0, 'pmerge', 0, False]
1300000
['finished

11300000
['finished', 393, 0, 0, 1, 0, 'panda-client-0.5.87-jedi-run', 0, False]
11400000
['finished', 322, 0, 0, 1, 0, 'panda-client-0.5.87-jedi-run', 0, False]
11500000
['finished', 693, 0, 0, 1, 0, 'panda-client-0.5.87-jedi-run', 0, False]
11600000
['finished', 1632, 0, 0, 1, 0, 'panda-client-0.5.87-jedi-run', 0, False]
11700000
['finished', 31054, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-athena', 0, False]
11800000
['closed', 0, 0, 0, 1, 0, 'evgen', 0, False]
11900000
['finished', 215, 0, 0, 1, 0, 'panda-client-0.5.87-jedi-run', 0, False]
12000000
['finished', 307, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-athena', 0, False]
12100000
['finished', 401, 0, 0, 1, 0, 'panda-client-0.5.87-jedi-athena', 0, False]
12200000
['finished', 7680, 0, 0, 1, 0, 'panda-client-0.5.85-jedi-run', 0, False]
12300000
['merging', 143584, 0, 0, 1, 0, 'simul', 0, False]
12400000
['finished', 149, 0, 0, 1, 0, 'panda-client-0.5.87-jedi-run', 0, False]
12500000
['finished', 718, 0, 0, 1, 0, 'panda-client-0.5.86-jed

22600000
['finished', 451, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-athena', 0, False]
22700000
['finished', 370, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
22800000
['finished', 138, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
22900000
['finished', 33878, 0, 0, 1, 0, 'evgen', 0, False]
23000000
['finished', 848480, 2, 0, 8, 0, 'simul', 2, False]
23100000
['finished', 14375, 0, 0, 1, 0, 'evgen', 0, False]
23200000
['finished', 37369, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
23300000
['finished', 1432, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
23400000
['finished', 756, 0, 0, 1, 0, 'gangarobot-pft', 0, False]
23500000
['finished', 1870, 0, 0, 1, 0, 'panda-client-0.5.85-jedi-athena', 0, False]
23600000
['finished', 28244, 0, 0, 1, 0, 'evgen', 0, False]
23700000
['finished', 767, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
23800000
['finished', 146, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
23900000
['finished', 36594, 0, 0,

### Count shared job

In [10]:
count = 0
nchainperactivity = {}
nchainperactivity_withshared = {}
for key,group in jobgroups.items():
    if not count%10000 or count < 100: 
        print(count)
    count += 1
    if group[6] not in nchainperactivity:
        nchainperactivity[group[6]] = 0
        nchainperactivity_withshared[group[6]] = 0
    nchainperactivity[group[6]] += 1
    if group[8]:
        nchainperactivity_withshared[group[6]] += 1

print('Number of chains')
print(nchainperactivity)
print('Number of chains with shared jobs')
print(nchainperactivity_withshared)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
10

10110000
10120000
10130000
10140000
10150000
10160000
10170000
10180000
10190000
10200000
10210000
10220000
10230000
10240000
10250000
10260000
10270000
10280000
10290000
10300000
10310000
10320000
10330000
10340000
10350000
10360000
10370000
10380000
10390000
10400000
10410000
10420000
10430000
10440000
10450000
10460000
10470000
10480000
10490000
10500000
10510000
10520000
10530000
10540000
10550000
10560000
10570000
10580000
10590000
10600000
10610000
10620000
10630000
10640000
10650000
10660000
10670000
10680000
10690000
10700000
10710000
10720000
10730000
10740000
10750000
10760000
10770000
10780000
10790000
10800000
10810000
10820000
10830000
10840000
10850000
10860000
10870000
10880000
10890000
10900000
10910000
10920000
10930000
10940000
10950000
10960000
10970000
10980000
10990000
11000000
11010000
11020000
11030000
11040000
11050000
11060000
11070000
11080000
11090000
11100000
11110000
11120000
11130000
11140000
11150000
11160000
11170000
11180000
11190000
11200000
11210000
1

19220000
19230000
19240000
19250000
19260000
19270000
19280000
19290000
19300000
19310000
19320000
19330000
19340000
19350000
19360000
19370000
19380000
19390000
19400000
19410000
19420000
19430000
19440000
19450000
19460000
19470000
19480000
19490000
19500000
19510000
19520000
19530000
19540000
19550000
19560000
19570000
19580000
19590000
19600000
19610000
19620000
19630000
19640000
19650000
19660000
19670000
19680000
19690000
19700000
19710000
19720000
19730000
19740000
19750000
19760000
19770000
19780000
19790000
19800000
19810000
19820000
19830000
19840000
19850000
19860000
19870000
19880000
19890000
19900000
19910000
19920000
19930000
19940000
19950000
19960000
19970000
19980000
19990000
20000000
20010000
20020000
20030000
20040000
20050000
20060000
20070000
20080000
20090000
20100000
20110000
20120000
20130000
20140000
20150000
20160000
20170000
20180000
20190000
20200000
20210000
20220000
20230000
20240000
20250000
20260000
20270000
20280000
20290000
20300000
20310000
20320000
2

28330000
28340000
28350000
28360000
28370000
28380000
28390000
28400000
28410000
28420000
28430000
28440000
28450000
28460000
28470000
28480000
28490000
28500000
28510000
28520000
28530000
28540000
28550000
28560000
28570000
28580000
28590000
28600000
28610000
28620000
28630000
28640000
28650000
28660000
28670000
28680000
28690000
28700000
28710000
28720000
28730000
28740000
28750000
28760000
28770000
28780000
28790000
28800000
28810000
28820000
28830000
28840000
28850000
28860000
28870000
28880000
28890000
28900000
28910000
28920000
28930000
28940000
28950000
28960000
28970000
28980000
28990000
29000000
29010000
29020000
29030000
29040000
29050000
29060000
29070000
29080000
29090000
29100000
29110000
29120000
29130000
29140000
29150000
29160000
29170000
29180000
29190000
29200000
29210000
29220000
29230000
29240000
29250000
29260000
29270000
29280000
29290000
29300000
29310000
29320000
29330000
29340000
29350000
29360000
29370000
29380000
29390000
29400000
29410000
29420000
29430000
2

### Print result

In [14]:
count = 0
count2 = 0
count3 = 0
data_finished = {}
data_failed = {}
data_closed = {}
data_cancelled = {}
data_merging = {}
data_unknown = {}
fill_number = True
for key,group in jobgroups.items():
    if group[8]:
        continue

    count += 1
    if group[6] not in data_finished:
        data_finished[group[6]] = []
        data_failed[group[6]] = []
        data_closed[group[6]] = []
        data_cancelled[group[6]] = []
        data_merging[group[6]] = []
        data_unknown[group[6]] = []
    if fill_number:
        if group[0] == "finished":
            data_finished[group[6]].append([group[2], 1])
        elif group[0] == "failed":
            data_failed[group[6]].append([group[2], 1])
            if group[6] == 'deriv' and group[2] == 20:
                count2 += 1
                if count2 < 100:
                    print(str(key)+' '+str(group[2])+' '+str(group[1])+' '+str(group[7]))
            if group[6] == 'simul':
                count3 += 1
                if count3 < 1000:
                    print('failed simul'+str(key)+' '+str(group[2])+' '+str(group[1])+' '+str(group[7]))
        elif group[0] == "closed":
            data_closed[group[6]].append([group[2], 1])
        elif group[0] == "cancelled":
            data_cancelled[group[6]].append([group[2], 1])
        elif group[0] == "merging":
            data_merging[group[6]].append([group[2], 1])
        else:
            data_unknown[group[6]].append([group[2], 1])

    else:
        if group[0] == "finished":
            data_finished[group[6]].append([group[2], group[1]])
        elif group[0] == "failed":
            data_failed[group[6]].append([group[2], group[1]])
        elif group[0] == "closed":
            data_closed[group[6]].append([group[2], group[1]])
        elif group[0] == "cancelled":
            data_cancelled[group[6]].append([group[2], group[1]])
        elif group[0] == "merging":
            data_merging[group[6]].append([group[2], group[1]])
        else:
            data_unknown[group[6]].append([group[2], group[1]])

    #data[group[6]].append([group[3], group[1]])
    if count < 10:
      print(group)

jobdata = {}
jobdatasum = {}
pd.set_option('display.max_rows', 1000)

print('=== finished ===')
for key,datum in data_finished.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])

print('=== failed ===')
for key,datum in data_failed.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== closed ===')
for key,datum in data_closed.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== cancelled ===')
for key,datum in data_cancelled.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== merging ===')
for key,datum in data_merging.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])
    
print('=== unknown ===')
for key,datum in data_unknown.items():
    jobdata[key] = pd.DataFrame(datum,columns=['depth', 'totalwalltime'])
    jobdata[key].head()
    job_depthgroup = jobdata[key].groupby('depth')
    jobdatasum[key] = job_depthgroup.sum()
    print(key)
    print(jobdatasum[key])

['finished', 342, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 253, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 209, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 330, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 399, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 496, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 432, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 169, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
['finished', 272, 0, 0, 1, 0, 'panda-client-0.5.86-jedi-run', 0, False]
failed simul3556775540 1 64928 1
failed simul3556786567 1 24128 1
failed simul3556786570 1 333408 1
failed simul3556786622 1 16408 1
failed simul3556786626 1 239960 1
failed simul3556786631 1 13080 1
failed simul3556786654 1 68076 1
failed simul3556786680 1 185376 1
failed simul3556786682 1 78402 1
failed simul3556786857 1 5496 1
failed simul35567868

failed simul3558153359 1 2392 1
failed simul3558153397 1 3520 1
failed simul3558153445 1 96462 1
failed simul3558153454 1 16596 1
failed simul3558153486 1 16116 1
failed simul3558153502 1 3192 1
failed simul3558153509 1 20070 1
failed simul3558153557 1 36024 1
failed simul3558153585 1 26400 1
failed simul3558153621 1 91184 1
failed simul3558153645 1 79128 1
failed simul3558153681 1 40672 1
failed simul3558188016 2 29968 2
failed simul3558205854 2 32128 2
failed simul3558206103 1 432 1
failed simul3558206113 1 432 1
failed simul3558206130 2 29296 2
failed simul3558206274 1 432 1
failed simul3558206282 1 432 1
failed simul3558206285 1 432 1
failed simul3558206290 1 432 1
failed simul3558206295 1 432 1
failed simul3558206300 1 432 1
failed simul3558206303 1 432 1
failed simul3558206306 1 432 1
failed simul3558206314 2 29808 2
failed simul3558219731 2 28832 2
failed simul3558219826 2 65440 2
failed simul3558219840 2 30880 2
failed simul3558237513 2 29968 2
failed simul3558252294 2 67104 2


failed simul3560878631 1 174082 1
failed simul3560878632 1 166874 1
failed simul3560878633 12 30572 12
failed simul3560878634 2 4105 2
failed simul3560878635 2 4449 2
failed simul3560878636 2 4349 2
failed simul3560878637 2 4344 2
failed simul3560878638 2 4061 2
failed simul3560878639 2 4493 2
failed simul3560878640 2 4324 2
failed simul3560878641 2 4504 2
failed simul3560878642 2 4567 2
failed simul3560878643 2 4083 2
failed simul3560878644 2 4655 2
failed simul3560878645 2 4671 2
failed simul3560878646 11 30029 11
failed simul3560878647 2 4745 2
failed simul3560878648 11 27369 11
failed simul3560878649 2 5779 2
failed simul3560878650 2 4196 2
failed simul3560878803 1 299 1
failed simul3560878804 1 304 1
failed simul3560878805 1 310 1
failed simul3560878806 1 649 1
failed simul3560878807 2 5476 2
failed simul3560878808 1 154480 1
failed simul3560878809 1 142057 1
failed simul3560878810 2 3567 2
failed simul3560879051 1 112704 1
failed simul3560879099 1 112860 1
failed simul3560879101 

gangarobot-rctest
       totalwalltime
depth               
0             822399
pmerge
       totalwalltime
depth               
0            1344695
1               1989
2                371
3                  3
6                  1
panda-client-0.5.86-jedi-athena
       totalwalltime
depth               
0            1951707
1              52663
2              12290
3               2611
4                317
5                116
6                 24
7                  5
8                 21
9                 48
10                11
11                14
12                 9
panda-client-0.5.85-jedi-athena
       totalwalltime
depth               
0              40858
1                473
2                280
3                615
5                 15
6                 12
panda-client-0.5.86-jedi-run-evp
       totalwalltime
depth               
0              33148
1                 67
2                 16
panda-client-0.5.80-jedi-athena-trf
Empty DataFrame
Columns: [totalwalltime]
Ind

simul
       totalwalltime
depth               
0            1133294
1              93288
2              20112
3               3500
4               1108
5                317
6                136
7                 81
8                 18
9                 13
10                29
11                 9
12                 4
13                 6
15                 5
16                 1
17                 3
18                 2
19                 1
20                 1
22                 1
24                 4
25                 1
27                 1
40                 1
47                 1
52                 1
53                 1
54                 1
55                 1
57                 2
58                 2
59                 1
60                 2
61                 1
62                 1
63                 1
64                 1
65                 2
67                 2
68                 1
75                 1
76                 1
89                 1
90                 1
93     

20                 1
panda-client-0.5.85-jedi-run
       totalwalltime
depth               
1               1205
2                557
3                485
4                472
5                356
6                 57
8                  6
9                  1
10                 5
30                 1
panda-client-0.5.86-jedi-run
       totalwalltime
depth               
1              66116
2              11337
3              74858
4                320
5                633
6                282
7                611
8               2288
9                 88
10               153
11                96
12               194
13               463
14                 4
15                45
16                 5
17                19
18                49
19                 1
20                 3
21                 4
22                 2
23                10
25                 3
26                 3
27                 1
28                 8
33                 7
34                 1
37                

panda-client-0.5.87-jedi-run
       totalwalltime
depth               
0              99137
deriv
       totalwalltime
depth               
0               4686
panda-client-0.5.85-jedi-run
       totalwalltime
depth               
0                596
panda-client-0.5.86-jedi-run
       totalwalltime
depth               
0             179934
panda-client-0.5.88-jedi-run
       totalwalltime
depth               
0               2327
gangarobot
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.87-jedi-athena-trf
       totalwalltime
depth               
0              10819
hammercloud
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.87-jedi-athena-evp
Empty DataFrame
Columns: [totalwalltime]
Index: []
None
       totalwalltime
depth               
0                524
panda-client-0.5.84-jedi-run
       totalwalltime
depth               
0                  3
panda-client-0.5.78-jedi-run
       totalwalltime
depth               
0                308


gangarobot
       totalwalltime
depth               
0              14725
1                  1
2                  1
3                  1
5                  1
7                  1
panda-client-0.5.87-jedi-athena-trf
       totalwalltime
depth               
0               4914
1                 22
2                 33
hammercloud
       totalwalltime
depth               
0               5757
panda-client-0.5.87-jedi-athena-evp
Empty DataFrame
Columns: [totalwalltime]
Index: []
None
       totalwalltime
depth               
0                  1
panda-client-0.5.84-jedi-run
       totalwalltime
depth               
0               1507
panda-client-0.5.78-jedi-run
       totalwalltime
depth               
0                 21
eventIndex
       totalwalltime
depth               
0                  4
1                  4
2                  1
3                  4
4                  3
5                  2
6                  2
7                  2
8                  2
panda-client-0.5.70-jedi

Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.84-jedi-athena-trf
Empty DataFrame
Columns: [totalwalltime]
Index: []
6.6.4-jedi-athena
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.83-jedi-athena
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.88-jedi-athena-trf
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.87-jedi-run-evp
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.38-jedi-run
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.80-jedi-run
Empty DataFrame
Columns: [totalwalltime]
Index: []
reprocessing
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.88-jedi-run-evp
Empty DataFrame
Columns: [totalwalltime]
Index: []
overlay
Empty DataFrame
Columns: [totalwalltime]
Index: []
gangarobot-celpft
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.90-jedi-athena
Empty DataFrame
Columns: [totalwalltime]
Index: []
panda-client-0.5.87-